In [1]:
import numpy as np
import pandas as pd
import os

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
training = pd.read_csv("matched_training_images.csv")

testing = pd.read_csv("matched_testing_images.csv")

validation = pd.read_csv("matched_validation_images.csv")

In [3]:
training.shape[0] + testing.shape[0] + validation.shape[0]

841

# Testing sets

In [4]:
testing_groups = pd.read_csv("/storage/data/DSB2018/metadata/image_groups_defined_by_beth.csv")
test_meta = pd.read_csv("/storage/data/DSB2018/metadata/dsb_summary_metadata_stage2testdata_v2.csv")

In [5]:
testing["Image"] = testing["crop"].str.split("/", expand=True)[5].str.replace(".png","")
testing = pd.merge(testing, testing_groups, on="Image")
merged_test = pd.merge(test_meta, testing, left_on="hash_name", right_on="Image")

In [6]:
merged_test.groupby(["project_id", "Group"])["Image","fluorescence"].count()

,,Image,fluorescence
project_id,Group,,
p_16ure_pig,Default,4,4
p_17Fraunhofer_carp,Default,8,8
p_17Fraunhofer_koi,Default,4,4
p_17Fraunhofer_sturgeon,Default,1,1
p_17Fraunhofer_trout,Default,8,8
p_17Marinaccio,Purple,16,16
p_17Moscow,Default,16,16
p_17_forum_cham,Default,1,1
p_17_forum_florian,Default,4,4


In [7]:
merged_test.to_csv("test_metadata.csv", index=False)

# Training sets

In [8]:
pink_purple = os.listdir("/data1/image-segmentation/dsb_eval/groups/my_groups/Pink-Purple/")
super_big = os.listdir("/data1/image-segmentation/dsb_eval/groups/my_groups/SuperBig/")
tissue_bw = os.listdir("/data1/image-segmentation/dsb_eval/groups/my_groups/TissueBW/")
purple = os.listdir("/data1/image-segmentation/dsb_eval/groups/my_groups/Purple/")
default = os.listdir("/data1/image-segmentation/dsb_eval/groups/my_groups/Default/")

In [9]:
default = list( set(default).difference(set(super_big)) )

In [10]:
len(pink_purple) + len(super_big) + len(tissue_bw) + len(purple) + len(default)

670

In [11]:
groups = ["PinkAndPurple"]*len(pink_purple) + ["SuperBig"]*len(super_big) + ["TissueBW"]*len(tissue_bw) + ["Purple"]*len(purple) + ["Default"]*len(default)
images = pink_purple + super_big + tissue_bw + purple + default
data = [{"Image":i, "Group":g} for i,g in zip(images, groups)]

In [12]:
training_groups = pd.DataFrame(data=data)

In [13]:
training["Image"] = training["crop"].str.split("/", expand=True)[7]
training["project"] = training.match.str.split("/", expand=True)[5]

In [14]:
training = pd.merge(training, training_groups, on="Image")

In [15]:
training.groupby(["project", "Group"]).count()

,,coord_x,coord_y,crop,exact,match,Image
project,Group,,,,,,
bbbc002,Default,72,72,72,72,72,72
bbbc018,Default,144,144,144,144,144,144
bbbc022,Default,43,43,43,43,43,43
p_06Eggert,Default,49,49,49,49,49,49
p_07mito,Default,5,5,5,5,5,5
p_07mito_fibro,Default,1,1,1,1,1,1
p_14ISBI,TissueBW,16,16,16,16,16,16
p_15Stachler_bw,Default,4,4,4,4,4,4
p_15Stachler_c1,Purple,66,66,66,66,66,66


In [16]:
training.to_csv("training_metadata.csv", index=False)